In [2]:
import torch
import numpy as np
import torch.nn as nn
from tqdm import tqdm_gui
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype=torch.FloatTensor

In [ ]:
sentences = ["jack like dog", "jack like cat", "jack like animal",
  "dog cat animal", "banana apple cat dog like", "dog fish milk like",
  "dog cat animal like", "jack like apple", "apple like", "jack like banana",
  "apple banana jack movie book music like", "cat dog hate", "cat dog like"]
senetences_list=" ".join(sentences).split()
vocab=list(set(senetences_list))
word2idx={w:i for i,w in enumerate(vocab)}
vocab_size=len(vocab)